<a href="https://colab.research.google.com/github/987-laura/masterschool-project-time-series/blob/main/Time_series_Project_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Daten und Bibliotheken laden

In [1]:
# Bibliotheken importieren

import pandas as pd
import numpy as np
#import requests
#import io
#import gdown

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Read data

df_holiday_events = pd.read_csv("/content/drive/MyDrive/Time-series-Projekt/holidays_events.csv")
df_items = pd.read_csv("/content/drive/MyDrive/Time-series-Projekt/items.csv")
df_oil = pd.read_csv("/content/drive/MyDrive/Time-series-Projekt/oil.csv")
#df_sample_submission = pd.read_csv("/content/drive/MyDrive/Time-series-Projekt/sample_submission.csv")
df_stores = pd.read_csv("/content/drive/MyDrive/Time-series-Projekt/stores.csv")
#df_test = pd.read_csv("/content/drive/MyDrive/Time-series-Projekt/test.csv")
df_transactions = pd.read_csv("/content/drive/MyDrive/Time-series-Projekt/transactions.csv")

In [4]:
# Note: train file is very large.
# To make sure we fit RAM avaible on Google-Colab, we will load only part of the file
df_train = pd.read_csv("/content/drive/MyDrive/Time-series-Projekt/train.csv", nrows=2e6)

In [5]:
store_ids = df_stores[df_stores['state'] == 'Guayas']['store_nbr'].unique()

In [6]:
# Initialize an empty list to hold filtered chunks
filtered_chunks = []

# Define the chunk size (number of rows per chunk)
chunk_size = 10 ** 6  # Adjust based on your system's memory capacity

# Read the CSV file in chunks
for chunk in pd.read_csv('/content/drive/MyDrive/Time-series-Projekt/train.csv', chunksize=chunk_size):
    # Filter the chunk for the desired store IDs
    chunk_filtered = chunk[chunk['store_nbr'].isin(store_ids)]
    # Append the filtered chunk to the list
    filtered_chunks.append(chunk_filtered)
    # Optional: Delete the chunk to free up memory
    del chunk

# Concatenate all filtered chunks into a single DataFrame
df_train = pd.concat(filtered_chunks, ignore_index=True)

# Clean up to free memory
del filtered_chunks

/tmp/ipython-input-17867064.py:8: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('/content/drive/MyDrive/Time-series-Projekt/train.csv', chunksize=chunk_size):


In [7]:
# Save df_train as a pickle file to avoid reloading it once colab disconnects

# 2.  Pick a folder inside Drive (create it if it doesn’t exist)
save_path = "/content/drive/MyDrive/Time-series-Projekt/df_train_original.pkl" # example of path

# 3.  SAVE the cleaned DataFrame
df_train.to_pickle(save_path)
print(f"Saved to: {save_path}")


Saved to: /content/drive/MyDrive/Time-series-Projekt/df_train_original.pkl


# Daten Überblick

In [8]:
df_train.shape

(22941656, 6)

In [9]:
# Loads first 5 rows
df_train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,18789,2013-01-02,24,103665,6.0,NaN
1,18790,2013-01-02,24,105574,12.0,NaN
2,18791,2013-01-02,24,105575,2.0,NaN
3,18792,2013-01-02,24,105577,1.0,NaN
4,18793,2013-01-02,24,105693,7.0,NaN


In [10]:
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [11]:
# Initiales Date-Parsing für alle DataFrames
# Konvertierung um spätere Fehler bei Joins oder Filtern zu vermeiden.
for df in [df_holiday_events, df_transactions, df_oil, df_items, df_stores, df_train]:
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])

In [12]:
print(df_train.info())
print()
print(df_holiday_events.info())
print()
print(df_items.info())
print()
print(df_oil.info())
print()
print(df_stores.info())
print(df_transactions.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22941656 entries, 0 to 22941655
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   item_nbr     int64         
 4   unit_sales   float64       
 5   onpromotion  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 1.0+ GB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         350 non-null    datetime64[ns]
 1   type         350 non-null    object        
 2   locale       350 non-null    object        
 3   locale_name  350 non-null    object        
 4   description  350 non-null    object        
 5   transferred  350 non-null    bool          
dtypes: bool(1), datetime64[ns](1), obj

# EDA

## Missing values

df_holiday_events
df_items          
df_oil            
df_stores         
df_train          
df_transactions   

In [13]:
# Checking missing values
print(f"\t df_train\n{df_train.isnull().sum()}\n")
print(f"\t df_holiday_events\n{df_holiday_events.isnull().sum()}\n")
print(f"\t df_items\n{df_items.isnull().sum()}\n")
print(f"\t df_oil\n{df_oil.isnull().sum()}\n")
print(f"\t df_stores\n{df_stores.isnull().sum()}\n")
print(f"\t df_transactions\n{df_transactions.isnull().sum()}")

	 df_train
id                   0
date                 0
store_nbr            0
item_nbr             0
unit_sales           0
onpromotion    3980341
dtype: int64

	 df_holiday_events
date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

	 df_items
item_nbr      0
family        0
class         0
perishable    0
dtype: int64

	 df_oil
date           0
dcoilwtico    43
dtype: int64

	 df_stores
store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

	 df_transactions
date            0
store_nbr       0
transactions    0
dtype: int64


In [14]:
# Focusing on missing values in the 'onpromotion' column
df_train['onpromotion'] = df_train['onpromotion'].fillna(False).astype(bool)

/tmp/ipython-input-2380528398.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['onpromotion'] = df_train['onpromotion'].fillna(False).astype(bool)


In [15]:
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].fillna(df_oil['dcoilwtico'].median())

In [16]:
# Checking missing values
df_train.isnull().sum()

,0
id,0
date,0
store_nbr,0
item_nbr,0
unit_sales,0
onpromotion,0


## Handling Outliers

### negative Sales bereinigen

In [17]:
# Checking for negative sales (returns)
negative_sales = df_train[df_train['unit_sales'] < 0 ]

negative_sales.head()  # Viewing negative sales for analysis

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
18589,106658,2013-01-04,34,586824,-2.0,False
54236,309805,2013-01-09,35,1079215,-8.0,False
64452,382905,2013-01-11,27,1097187,-20.0,False
87113,507901,2013-01-14,30,959437,-34.0,False
93833,547212,2013-01-15,30,812751,-168.0,False


In [18]:
# Replacing negative sales with 0 to reflect returns as non-sales
df_train['unit_sales'] = df_train['unit_sales'].apply(lambda x: max(x, 0))

In [19]:
# Checking negative sales got correctly replaced
df_train[df_train['unit_sales'] < 0]

,id,date,store_nbr,item_nbr,unit_sales,onpromotion


### Ausreißer via Z-Score finden

In [20]:
"""
    Berechnet den Z-Score für die 'unit_sales' innerhalb jeder Gruppe (store_nbr, item_nbr).
    Der Z-Score gibt an, wie viele Standardabweichungen ein Datenpunkt vom Mittelwert entfernt ist.
    Eine Best Practice ist es, Division durch Null zu vermeiden, falls die Standardabweichung 0 ist
    (z.B. wenn alle Verkäufe für eine Gruppe gleich sind)."""


# Function to calculate Z-score for each group (store-item combination)
def calculate_store_item_zscore(group):
    # Compute mean and standard deviation for each store-item group
    mean_sales = group['unit_sales'].mean()
    std_sales = group['unit_sales'].std()

    # Calculate Z-score (avoiding division by zero for standard deviation)
    group['z_score'] = (group['unit_sales'] - mean_sales) / (std_sales if std_sales != 0 else 1)
    return group

# Apply the Z-score calculation to each store-item group,then flatten the index
df_train_grouped = df_train.groupby(['store_nbr', 'item_nbr']).apply(calculate_store_item_zscore)
df_train_grouped.reset_index(drop=True, inplace=True)

# Define threshold for outliers (e.g., Z-score > 5)
outliers = df_train_grouped[df_train_grouped['z_score'] > 5]

# Print summary
print(f"Number of outliers detected: {len(outliers)}")
outliers.head()

Number of outliers detected: 66817


/tmp/ipython-input-943745576.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train_grouped = df_train.groupby(['store_nbr', 'item_nbr']).apply(calculate_store_item_zscore)


,id,date,store_nbr,item_nbr,unit_sales,onpromotion,z_score
188,15666702,2013-12-18,24,96995,14.0,False,7.946399
286,59775862,2015-10-20,24,99197,14.0,False,5.822016
289,60322807,2015-10-26,24,99197,19.0,False,8.276173
721,15717753,2013-12-19,24,103520,27.0,False,6.642670
770,18886543,2014-02-13,24,103520,22.0,False,5.154400


### Werbung rausfiltern

In [21]:
# Vereinfachte und effizientere Filterung
outliers_notonpromotion = outliers[outliers['onpromotion'] == False]

# Überprüfen Sie das Ergebnis
print(f"Anzahl der Ausreißer (nicht auf Promotion) entdeckt: {len(outliers_notonpromotion)}")
print(outliers_notonpromotion.head())

Anzahl der Ausreißer (nicht auf Promotion) entdeckt: 49719
           id       date  store_nbr  item_nbr  unit_sales  onpromotion  \
188  15666702 2013-12-18         24     96995        14.0        False   
286  59775862 2015-10-20         24     99197        14.0        False   
289  60322807 2015-10-26         24     99197        19.0        False   
721  15717753 2013-12-19         24    103520        27.0        False   
770  18886543 2014-02-13         24    103520        22.0        False   

      z_score  
188  7.946399  
286  5.822016  
289  8.276173  
721  6.642670  
770  5.154400  


### Ferien rausfiltern

In [22]:
# Filtern nach Feiertagen
# Wir überprüfen, ob die Daten dieser Ausreißer mit Feiertagen übereinstimmen.
outliers_notonpromotion['is_holiday'] = outliers_notonpromotion['date'].isin(df_holiday_events['date'])

# Herausfiltern der Ausreißer, die auf Feiertage fallen
# endgültiges DataFrame mit den relevantesten Ausreißern.
outliers_notonpromotion_noholiday = outliers_notonpromotion[outliers_notonpromotion['is_holiday'] == False]

# --- 4. Zusammenfassung ausgeben ---
print(f"Anzahl der erkannten Ausreißer (nach allen Kriterien): {len(outliers_notonpromotion_noholiday)}")
print("\nKopf des DataFrames der relevanten Ausreißer:")
print(outliers_notonpromotion_noholiday.head())

Anzahl der erkannten Ausreißer (nach allen Kriterien): 33773

Kopf des DataFrames der relevanten Ausreißer:
           id       date  store_nbr  item_nbr  unit_sales  onpromotion  \
188  15666702 2013-12-18         24     96995        14.0        False   
286  59775862 2015-10-20         24     99197        14.0        False   
289  60322807 2015-10-26         24     99197        19.0        False   
721  15717753 2013-12-19         24    103520        27.0        False   
770  18886543 2014-02-13         24    103520        22.0        False   

      z_score  is_holiday  
188  7.946399       False  
286  5.822016       False  
289  8.276173       False  
721  6.642670       False  
770  5.154400       False  


/tmp/ipython-input-1694940874.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outliers_notonpromotion['is_holiday'] = outliers_notonpromotion['date'].isin(df_holiday_events['date'])


In [23]:
outliers_notonpromotion_noholiday.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33773 entries, 188 to 22940841
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           33773 non-null  int64         
 1   date         33773 non-null  datetime64[ns]
 2   store_nbr    33773 non-null  int64         
 3   item_nbr     33773 non-null  int64         
 4   unit_sales   33773 non-null  float64       
 5   onpromotion  33773 non-null  bool          
 6   z_score      33773 non-null  float64       
 7   is_holiday   33773 non-null  bool          
dtypes: bool(2), datetime64[ns](1), float64(2), int64(3)
memory usage: 1.9 MB


##Winsorisierung

In [24]:
# Schritt 1: Erstelle ein einfaches DataFrame, das nur die eindeutigen Feiertagsdaten enthält
holiday_dates = pd.DataFrame({
    'date': df_holiday_events['date'].unique(),
    'is_holiday': True
})

# Schritt 2: Führe einen Left Join durch, um die 'is_holiday' Spalte zu df_train hinzuzufügen
df_train = pd.merge(
    left=df_train,
    right=holiday_dates,
    on='date',
    how='left'
)

# Schritt 3: Fülle die NaN-Werte mit False
# Jeder Tag, der kein Feiertag ist, erhält den Wert False
df_train['is_holiday'] = df_train['is_holiday'].fillna(False)

# Überprüfen der neuen Spalte
print(df_train.head())
print(df_train['is_holiday'].value_counts())

      id       date  store_nbr  item_nbr  unit_sales  onpromotion  is_holiday
0  18789 2013-01-02         24    103665         6.0        False       False
1  18790 2013-01-02         24    105574        12.0        False       False
2  18791 2013-01-02         24    105575         2.0        False       False
3  18792 2013-01-02         24    105577         1.0        False       False
4  18793 2013-01-02         24    105693         7.0        False       False
is_holiday
False    19409937
True      3531719
Name: count, dtype: int64


/tmp/ipython-input-220741851.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['is_holiday'] = df_train['is_holiday'].fillna(False)


### Data Gliches finden

Analyse der neuen Ergebnisse 📊

Ihre neuen Diagramme zeigen nicht nur "leere" Daten, sondern klarere Verkaufsspitzen, die direkt auf Datenfehler hindeuten. Es gibt zwei Hauptmuster, die Sie beobachten können:

Das "Glitch"-Muster (Bild 3, 4, 5): Dieses Muster ist das, wonach wir gesucht haben. Es zeigt eine einzelne, massive Verkaufsspitze, die von Tagen mit Verkäufen nahe Null umgeben ist. Es ist sehr unwahrscheinlich, dass dies ein echtes Verhaltensmuster ist.

Bild 3: Verkäufe sind für mehrere Tage bei Null, springen dann am 17. Januar 2017 auf über 4.800 und fallen sofort wieder auf Null.

Bild 4: Ähnliches Muster mit einem Höhepunkt am 15. Dezember 2014.

Bild 5: Der Verkauf springt am 6. Januar 2017 auf über 5.500 und fällt danach wieder auf Null.
Diese Muster deuten stark darauf hin, dass die Verkäufe von mehreren Tagen in einer einzigen Transaktion zusammengefasst wurden, was wahrscheinlich auf einen technischen Fehler bei der Datenverarbeitung zurückzuführen ist.

Das "Dreieck"-Muster (Bild 1 und 2): Dieses Muster zeigt einen sanften Anstieg zu einem Höhepunkt und einen ebenso sanften Abfall.

Bild 1: Der Verkauf steigt über einige Tage hinweg stetig an und erreicht am 17. Januar 2017 einen Höhepunkt, bevor er abrupt wieder auf null fällt.

Bild 2: Hier sehen Sie eine massive Verkaufsspitze an einem einzigen Tag, gefolgt von Tagen ohne Verkäufe. Dies ist ein selteneres Muster, kann aber immer noch auf einen Datenfehler oder ein einmaliges Großereignis (z. B. einen Großauftrag) hinweisen. Da die Nachfrage nach diesem Artikel danach auf Null fällt, ist ein Datenfehler wahrscheinlicher.

Fazit und Nächste Schritte 🚀

Die Visualisierung hat ihre Aufgabe erfüllt: Sie haben die Arten von Ausreißern in Ihren Daten identifiziert. Da die meisten dieser Ausreißer höchstwahrscheinlich auf Datenprobleme zurückzuführen sind, ist das Winsorizing, das wir zuvor besprochen haben, die richtige Strategie. Es wird diese extremen Spitzen auf ein realistischeres Niveau begrenzen, was Ihr Modell vor der Überanpassung an fehlerhafte Daten schützt.

In [25]:
# funktioniert
'''
# Untersuchung potenzieller Daten-Glitches

# Filtern der Ausreißer, die am interessantesten für diese Hypothese sind:
# Hohe positive Ausreißer (bereits durch z_score > 5 erfasst)
# die nicht auf Promotion waren und nicht auf Feiertage fallen.
# Wir müssen df_train um die Z-Scores erweitern, um hier richtig filtern zu können.
# Da df_train_grouped bereits die Z-Scores enthält, nutzen wir dieses DataFrame.

# Betrachte die 5 größten Ausreißer, die nicht auf Promotion und keine Feiertage sind.
# Diese sind gute Kandidaten, um die Hypothese zu überprüfen.
potential_glitch_outliers = outliers_notonpromotion_noholiday.nlargest(5, 'unit_sales')

print("\nTop 5 potenzielle Glitch-Ausreißer (nicht auf Promotion, keine Feiertage):")
print(potential_glitch_outliers)

# Für jeden dieser Top-Ausreißer, visualisiere die Verkäufe rund um das Datum
for index, row in potential_glitch_outliers.iterrows():
    store = row['store_nbr']
    item = row['item_nbr']
    outlier_date = pd.to_datetime(row['date'])

    # Definiere einen Zeitrahmen um das Ausreißerdatum herum (z.B. +/- 7 Tage)
    start_date = outlier_date - pd.Timedelta(days=7)
    end_date = outlier_date + pd.Timedelta(days=7)

    # Filtere die Verkaufsdaten für diese spezifische store_nbr und item_nbr
    # und den definierten Zeitrahmen
    subset_data = df_train[(df_train['store_nbr'] == store) &
                           (df_train['item_nbr'] == item) &
                           (df_train['date'] >= start_date) &
                           (df_train['date'] <= end_date)].sort_values('date')

    # Erstelle ein Liniendiagramm
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=subset_data, x='date', y='unit_sales', marker='o')
    plt.axvline(outlier_date, color='red', linestyle='--', label='Ausreißer-Datum') # Markiere das Ausreißer-Datum

    # Füge eine Markierung für das Datum VOR dem Ausreißer hinzu
    date_before_outlier = outlier_date - pd.Timedelta(days=1)
    if date_before_outlier in subset_data['date'].values:
        plt.axvline(date_before_outlier, color='green', linestyle=':', label='Tag vor Ausreißer')

    plt.title(f'Verkäufe für Store {store}, Item {item} um {outlier_date.strftime("%Y-%m-%d")}')
    plt.xlabel('Datum')
    plt.ylabel('Unit Sales')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Überprüfe die Daten am Tag vor dem Ausreißer manuell
    # Dies ist der entscheidende Schritt, um die Hypothese zu bestätigen
    sales_day_before = subset_data[subset_data['date'] == date_before_outlier]
    if not sales_day_before.empty:
        print(f"Verkäufe am Tag vor dem Ausreißer ({date_before_outlier.strftime('%Y-%m-%d')}):")
        print(sales_day_before)
        if sales_day_before['unit_sales'].iloc[0] == 0 and negative_sales_before_correction[(negative_sales_before_correction['store_nbr'] == store) &
                                                                                               (negative_sales_before_correction['item_nbr'] == item) &
                                                                                               (negative_sales_before_correction['date'] == date_before_outlier)].any().any():
            print(f"⚠️ Achtung: Am Tag vor dem Ausreißer ({date_before_outlier.strftime('%Y-%m-%d')}) gab es ursprünglich negative Verkäufe (Retouren/Stornierungen), die auf 0 gesetzt wurden. Dies könnte ein 'Glitch' sein.")
    print("-" * 50)
    '''

'\n# Untersuchung potenzieller Daten-Glitches\n\n# Filtern der Ausreißer, die am interessantesten für diese Hypothese sind:\n# Hohe positive Ausreißer (bereits durch z_score > 5 erfasst)\n# die nicht auf Promotion waren und nicht auf Feiertage fallen.\n# Wir müssen df_train um die Z-Scores erweitern, um hier richtig filtern zu können.\n# Da df_train_grouped bereits die Z-Scores enthält, nutzen wir dieses DataFrame.\n\n# Betrachte die 5 größten Ausreißer, die nicht auf Promotion und keine Feiertage sind.\n# Diese sind gute Kandidaten, um die Hypothese zu überprüfen.\npotential_glitch_outliers = outliers_notonpromotion_noholiday.nlargest(5, \'unit_sales\')\n\nprint("\nTop 5 potenzielle Glitch-Ausreißer (nicht auf Promotion, keine Feiertage):")\nprint(potential_glitch_outliers)\n\n# Für jeden dieser Top-Ausreißer, visualisiere die Verkäufe rund um das Datum\nfor index, row in potential_glitch_outliers.iterrows():\n    store = row[\'store_nbr\']\n    item = row[\'item_nbr\']\n    outlier_d

### (fehlt) Data Gliches behandeln

Was ist **Winsorisierung** und warum ist sie besser? 🧢

Bei der Winsorisierung werden alle Datenpunkte, die über einem bestimmten Perzentil (z.B. dem 99. Perzentil) liegen, auf den Wert dieses Perzentils gesetzt. Das hat zwei große Vorteile:

Robustheit: Es macht deine Daten resistenter gegen die extremsten Ausreißer.

Datenerhalt: Du verlierst keine Informationen, da die Datenpunkte nicht gelöscht werden. Ein hoher Verkauf bleibt ein hoher Verkauf, wird aber als "maximal normal" behandelt.

In [26]:
'''
df_train_with_outliers = pd.merge(
    left=df_train,
    right=outliers_notonpromotion_noholiday,
    on=['date', 'store_nbr', 'item_nbr'],
    how='left'
)
'''

"\ndf_train_with_outliers = pd.merge(\n    left=df_train,\n    right=outliers_notonpromotion_noholiday,\n    on=['date', 'store_nbr', 'item_nbr'],\n    how='left'\n)\n"

In [27]:
# funktioniert nicht
'''# Wir verwenden das DataFrame df_train, das bereits bereinigt wurde.
# Wir müssen die Winsorisierung auf die Original-Verkaufsdaten anwenden.

# Funktion zur Winsorisierung auf Gruppenebene
def winsorize_group(group, percentile=0.99):
    """
    Begrenzt die Verkaufsdaten auf Gruppenebene (store_nbr, item_nbr)
    durch das Festlegen einer Obergrenze basierend auf einem Perzentil.
    """
    # Den Schwellenwert (upper_cap) für die Gruppe berechnen.
    # np.clip stellt sicher, dass der Wert nicht unter 0 fällt.
    upper_cap = group['unit_sales'].quantile(percentile)

    # Die Werte der Spalte 'unit_sales' auf die Obergrenze begrenzen.
    group['unit_sales_winsorized'] = np.clip(group['unit_sales'], a_min=0, a_max=upper_cap)

    return group

# Den Winsorizing-Prozess auf das gesamte DataFrame anwenden
# Mit `group_keys=False` stellen wir sicher, dass die Gruppenschlüssel
# nicht als Teil des Indexes im Ergebnis-DataFrame erscheinen.
df_train_winsorized = df_train_with_outliers.groupby(['store_nbr', 'item_nbr'], group_keys=False).apply(winsorize_group)

# Vergleiche die Original-Verkaufszahlen mit den winsorisierten Zahlen
print("Original- und winsorisierte Verkaufszahlen:")
print(df_train_winsorized[['date', 'store_nbr', 'item_nbr', 'unit_sales', 'unit_sales_winsorized']].head())
print("-" * 50)


# Überprüfe einen Ausreißer
print("Details eines Ausreißers:")
print("Originalwert:", df_train_winsorized.loc[2, 'unit_sales'])
print("Winsorisierter Wert:", df_train_winsorized.loc[2, 'unit_sales_winsorized'])'''

'# Wir verwenden das DataFrame df_train, das bereits bereinigt wurde.\n# Wir müssen die Winsorisierung auf die Original-Verkaufsdaten anwenden.\n\n# Funktion zur Winsorisierung auf Gruppenebene\ndef winsorize_group(group, percentile=0.99):\n    """\n    Begrenzt die Verkaufsdaten auf Gruppenebene (store_nbr, item_nbr)\n    durch das Festlegen einer Obergrenze basierend auf einem Perzentil.\n    """\n    # Den Schwellenwert (upper_cap) für die Gruppe berechnen.\n    # np.clip stellt sicher, dass der Wert nicht unter 0 fällt.\n    upper_cap = group[\'unit_sales\'].quantile(percentile)\n\n    # Die Werte der Spalte \'unit_sales\' auf die Obergrenze begrenzen.\n    group[\'unit_sales_winsorized\'] = np.clip(group[\'unit_sales\'], a_min=0, a_max=upper_cap)\n\n    return group\n\n# Den Winsorizing-Prozess auf das gesamte DataFrame anwenden\n# Mit `group_keys=False` stellen wir sicher, dass die Gruppenschlüssel\n# nicht als Teil des Indexes im Ergebnis-DataFrame erscheinen.\ndf_train_winsoriz

##Fill missing dates

In [28]:
def fill_calendar(group):
    #
    # group contains all rows for ONE (store_nbr, item_nbr) pair
    #
    g = group.set_index("date").sort_index()   # use date/calendar as the index
    g = g.asfreq("D", fill_value=0)            # make it daily; add 0 where missing
    # put the identifiers back (asfreq drops them)
    g["store_nbr"] = group["store_nbr"].iloc[0]
    g["item_nbr"]  = group["item_nbr"].iloc[0]

    return g.reset_index()                     # date back to a normal column

In [29]:
df_train = (
    df_train
    .groupby(["store_nbr", "item_nbr"], group_keys=False)  # keeps memory low
    .apply(fill_calendar)
)

/tmp/ipython-input-1191857923.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(fill_calendar)


In [30]:
df_train.head()

,date,id,store_nbr,item_nbr,unit_sales,onpromotion,is_holiday
0,2013-01-09,302230,24,96995,2.0,False,False
1,2013-01-10,0,24,96995,0.0,0,0
2,2013-01-11,0,24,96995,0.0,0,0
3,2013-01-12,419989,24,96995,2.0,False,True
4,2013-01-13,0,24,96995,0.0,0,0


##Feature Engineering

In [31]:
# Make sure 'date' is a real datetime
df_train['date'] = pd.to_datetime(df_train['date'])

# Split the timestamp into model-friendly parts
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month
df_train['day'] = df_train['date'].dt.day
df_train['day_of_week'] = df_train['date'].dt.dayofweek # Monday=0 … Sunday=6

# Lets check the result
df_train.head()

,date,id,store_nbr,item_nbr,unit_sales,onpromotion,is_holiday,year,month,day,day_of_week
0,2013-01-09,302230,24,96995,2.0,False,False,2013,1,9,2
1,2013-01-10,0,24,96995,0.0,0,0,2013,1,10,3
2,2013-01-11,0,24,96995,0.0,0,0,2013,1,11,4
3,2013-01-12,419989,24,96995,2.0,False,True,2013,1,12,5
4,2013-01-13,0,24,96995,0.0,0,0,2013,1,13,6


In [32]:
# 1. Mount Google Drive (falls noch nicht geschehen)
from google.colab import drive
drive.mount('/content/drive')

# 2. Definiere den Speicherpfad
save_path = "/content/drive/MyDrive/Time-series-Projekt/df_train_clean.pkl"

# 3. Speichere das Ergebnis in der Datei
# Hinweis: Sie müssen hier das Chunken neu durchführen,
# da wir das Ergebnis nicht im RAM speichern können.
chunk_size = 10**6
first_chunk = True

# Öffne die Datei im 'write' Modus ('wb' für Pickle)
with open(save_path, 'wb') as f:
    for start in range(0, len(df_train), chunk_size):
        end = start + chunk_size
        chunk = df_train.iloc[start:end]

        # Berechne den 7-Tage-Rolling-Average
        chunk["unit_sales_7d_avg"] = (
            chunk
            .groupby(["item_nbr", "store_nbr"])["unit_sales"]
            .transform(lambda s: s.rolling(window=7, min_periods=1).mean())
        )

        # Speichere jeden Chunk einzeln
        # Fügen Sie hier die Winsorisierung hinzu, um den Prozess zu optimieren
        upper_caps = chunk.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform(lambda x: x.quantile(0.99))
        chunk['unit_sales_winsorized'] = np.clip(chunk['unit_sales'], a_min=0, a_max=upper_caps)

        # Schreiben Sie den Chunk in die Datei
        if first_chunk:
            # Für den ersten Chunk fügen Sie den Header hinzu
            # In diesem Fall ist es der erste DataFrame selbst
            chunk.to_pickle(f)
            first_chunk = False
        else:
            # Hängen Sie die nachfolgenden Chunks an die Datei an
            chunk.to_pickle(f, protocol=4) # protocol=4 ist für große Dateien gut geeignet

print(f"Das DataFrame wurde erfolgreich in Chunks unter: {save_path} gespeichert.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-619942036.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["unit_sales_7d_avg"] = (
/tmp/ipython-input-619942036.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['unit_sales_winsorized'] = np.clip(chunk['unit_sales'], a_min=0, a_max=upper_caps)
/tmp/ipython-input-619942036.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

Das DataFrame wurde erfolgreich in Chunks unter: /content/drive/MyDrive/Time-series-Projekt/df_train_clean.pkl gespeichert.


/tmp/ipython-input-619942036.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['unit_sales_winsorized'] = np.clip(chunk['unit_sales'], a_min=0, a_max=upper_caps)


In [33]:
'''
# 7-day rolling average of unit_sales, per (item, store)
df_train = df_train.sort_values(["item_nbr", "store_nbr", "date"]).reset_index(drop=True) # make sure rows are in time order

df_train["unit_sales_7d_avg"] = (
    df_train
    .groupby(["item_nbr", "store_nbr"])["unit_sales"]      # isolate one time-series per (item, store), get the units sold
    .transform(lambda s: s.rolling(window=7, min_periods=1).mean())       #  mean of last 7 days, i.e. 7-day moving average, aligned back to original df
)
'''

'\n# 7-day rolling average of unit_sales, per (item, store)\ndf_train = df_train.sort_values(["item_nbr", "store_nbr", "date"]).reset_index(drop=True) # make sure rows are in time order\n\ndf_train["unit_sales_7d_avg"] = (\n    df_train\n    .groupby(["item_nbr", "store_nbr"])["unit_sales"]      # isolate one time-series per (item, store), get the units sold\n    .transform(lambda s: s.rolling(window=7, min_periods=1).mean())       #  mean of last 7 days, i.e. 7-day moving average, aligned back to original df\n)\n'

In [34]:
'''
# Lets see how the new column unit_sales_7d_avg looks like. For that, we'll need to select a store and item.
# Get store and item from the first row
store_id = df_train.iloc[0]['store_nbr']
item_id = df_train.iloc[0]['item_nbr']

# Filter the DataFrame for this store-item pair
sample = df_train[(df_train['store_nbr'] == store_id) & (df_train['item_nbr'] == item_id)].sort_values("date")
sample.head()
'''

'\n# Lets see how the new column unit_sales_7d_avg looks like. For that, we\'ll need to select a store and item.\n# Get store and item from the first row\nstore_id = df_train.iloc[0][\'store_nbr\']\nitem_id = df_train.iloc[0][\'item_nbr\']\n\n# Filter the DataFrame for this store-item pair\nsample = df_train[(df_train[\'store_nbr\'] == store_id) & (df_train[\'item_nbr\'] == item_id)].sort_values("date")\nsample.head()\n'

## df_train_clean pickle

In [35]:
'''
# Save df_train as a pickle file to avoid reloading it once colab disconnects
# 1.  Mount Drive (one-time click to authorize)
#from google.colab import drive
#drive.mount('/content/drive')

# 2.  Pick a folder inside Drive (create it if it doesn’t exist)
save_path = "/content/drive/MyDrive/Time-series-Projekt/df_train_clean.pkl" # example of path

# 3.  SAVE the cleaned DataFrame
df_train.to_pickle(save_path)
print(f"Saved to: {save_path}")
'''

'\n# Save df_train as a pickle file to avoid reloading it once colab disconnects\n# 1.  Mount Drive (one-time click to authorize)\n#from google.colab import drive\n#drive.mount(\'/content/drive\')\n\n# 2.  Pick a folder inside Drive (create it if it doesn’t exist)\nsave_path = "/content/drive/MyDrive/Time-series-Projekt/df_train_clean.pkl" # example of path\n\n# 3.  SAVE the cleaned DataFrame\ndf_train.to_pickle(save_path)\nprint(f"Saved to: {save_path}")\n'